Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  

import pandas as pd
import numpy as np
import gc
from tqdm.notebook import tqdm

from transformers import *
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [2]:
%%time
df = pd.read_csv('/home/pinto/training.tsv', sep='\x01', header=None, usecols=[0] )
df.columns = ['text_tokens']
df['id']   = np.arange( df.shape[0] )
df['id']   = df['id'].astype(np.uint32)
gc.collect()
df.head()

CPU times: user 7min 56s, sys: 50.5 s, total: 8min 47s
Wall time: 15min 2s


,text_tokens,id
0,101\t1894\t47271\t11369\t82791\t39675\t34065\t...,0
1,101\t17607\t11382\t14444\t10499\t17025\t10854\...,1
2,101\t21333\t34755\t20338\t19554\t90582\t42588\...,2
3,101\t1910\t11588\t37311\t89744\t111791\t15221\...,3
4,101\t56898\t137\t38329\t25285\t13264\t11447\t5...,4


In [3]:
%%time
df['tweet'] = [ tokenizer.decode( [ int(n) for n in t.split('\t') ] ) for t in tqdm(df.text_tokens.values) ] 
gc.collect()


CPU times: user 2h 32min 33s, sys: 1min 23s, total: 2h 33min 57s
Wall time: 2h 32min 59s


15

In [4]:
%%time

df['tweet'] = df['tweet'].apply( lambda x: x.replace('https : / / t. co / ', 'https://t.co/') )
df['tweet'] = df['tweet'].apply( lambda x: x.replace('@ ', '@') )
df.head()

CPU times: user 3min 27s, sys: 14.4 s, total: 3min 41s
Wall time: 3min 41s


,text_tokens,id,tweet
0,101\t1894\t47271\t11369\t82791\t39675\t34065\t...,0,[CLS] 【 ALL JAPAN SINGLES TOP 20 】.. 〜 今 週 の１ ...
1,101\t17607\t11382\t14444\t10499\t17025\t10854\...,1,[CLS] Aonde eu deixo currículo pra isso? Hehe ...
2,101\t21333\t34755\t20338\t19554\t90582\t42588\...,2,"[CLS] Bagaimanapun kondisi anda saat ini, berb..."
3,101\t1910\t11588\t37311\t89744\t111791\t15221\...,3,[CLS] おはようございます 。 毎 日 twitterに 流 れてくる 各 国 のファン...
4,101\t56898\t137\t38329\t25285\t13264\t11447\t5...,4,[CLS] RT @PiyushGoyal : Take a look at the bea...


In [5]:
df_size=df.shape[0]
print(df_size)

106254462


In [6]:
df.to_parquet( 'train-tweet-1.parquet' )

In [7]:
gc.collect()
#del df

18

In [8]:
%%time
dv = pd.read_csv('/home/pinto/val.tsv', sep='\x01', header=None, usecols=[0] )
dv.columns = ['text_tokens']
dv['id']   = np.arange( df_size , df_size+dv.shape[0] ) 
dv['id']   = dv['id'].astype(np.uint32)
gc.collect()
dv.head()

CPU times: user 44.8 s, sys: 2.36 s, total: 47.1 s
Wall time: 1min 5s


,text_tokens,id
0,101\t10117\t140\t119\t142\t119\t152\t119\t1010...,106254462
1,101\t10105\t10817\t10124\t59232\t18121\t15629\...,106254463
2,101\t48561\t10116\t67737\t18554\t36371\t10989\...,106254464
3,101\t100055\t69940\t10414\t159\t11305\t11166\t...,106254465
4,101\t62154\t32221\t71843\t10143\t10237\t15507\...,106254466


In [10]:
%%time
dv['tweet'] = [ tokenizer.decode( [ int(n) for n in t.split('\t') ] ) for t in tqdm(dv.text_tokens.values) ] 
gc.collect()


CPU times: user 14min 5s, sys: 7.05 s, total: 14min 12s
Wall time: 14min 7s


15

In [14]:
%%time
dv['tweet'] = dv['tweet'].apply( lambda x: x.replace('https : / / t. co / ', 'https://t.co/') )
dv['tweet'] = dv['tweet'].apply( lambda x: x.replace('@ ', '@') )
dv.head()

CPU times: user 18 s, sys: 340 ms, total: 18.4 s
Wall time: 18.4 s


,text_tokens,id,tweet
0,101\t10117\t140\t119\t142\t119\t152\t119\t1010...,106254462,"[CLS] The C. E. O. of Acronym, the tech consul..."
1,101\t10105\t10817\t10124\t59232\t18121\t15629\...,106254463,[CLS] the man is literally barking.. [SEP]
2,101\t48561\t10116\t67737\t18554\t36371\t10989\...,106254464,"[CLS] Usai juara dunia enam tahun lalu, timnas..."
3,101\t100055\t69940\t10414\t159\t11305\t11166\t...,106254465,[CLS] Här hör ni V75 - podden med Niclas och E...
4,101\t62154\t32221\t71843\t10143\t10237\t15507\...,106254466,"[CLS] Bursa trfr dah ditutup, lo ngapain @juve..."


In [15]:
dv.to_parquet( 'valid-tweet-1.parquet' )

In [ ]:
%%time
dt = pd.read_csv('/home/pinto/test.tsv', sep='\x01', header=None, usecols=[0] )
dt.columns = ['text_tokens']
dt['id']   = np.arange( df_size+dv.shape[0], df_size+dv.shape[0]+dt.shape[0] )
dt['id']   = dt['id'].astype(np.uint32)
gc.collect()
dt.head()

In [11]:
%%time
dt['tweet'] = [ tokenizer.decode( [ int(n) for n in t.split('\t') ] ) for t in tqdm(dt.text_tokens.values) ] 
gc.collect()

KeyboardInterrupt: 

In [13]:
%%time
dt['tweet'] = dt['tweet'].apply( lambda x: x.replace('https : / / t. co / ', 'https://t.co/') )
dt['tweet'] = dt['tweet'].apply( lambda x: x.replace('@ ', '@') )

dt.head()

CPU times: user 21 s, sys: 276 ms, total: 21.2 s
Wall time: 21.2 s


,text_tokens,id,tweet
0,101\t3100\t5477\t3028\t4348\t1924\t111806\t186...,163202922,[CLS] 埼 玉 土 日 じゃん ！. 行 けないこともないか... ？ [UNK]. え...
1,101\t56898\t137\t36110\t10400\t168\t64062\t131...,163202923,[CLS] RT @meanie _ ark : いいね ・ RTでそれぞれポイントになるみ...
2,101\t56898\t137\t179\t36816\t10775\t40546\t513...,163202924,[CLS] RT @kwonjiyongbabe : this vip be hanging...
3,101\t56898\t137\t22038\t40663\t12892\t45389\t1...,163202925,[CLS] RT @massudessu13 : 너무좋아 [UNK].. # 박유천 ht...
4,101\t56898\t137\t11699\t10174\t10738\t37816\t1...,163202926,[CLS] RT @AtkArena : Welcome to our new CS : G...


In [14]:
dt.to_parquet( 'test-tweet-1.parquet' )